# Constraint satisfaction problem

In the previous tasks the state was opaque: an algorithm could execute an action and transform a state to another state, but that was it. It could not depend in its search on the details of a state, because the state representation was different from problem to problem. In CSP this is no longer true: the state always consists of an assignment of values over some domains to variables. This enables an algorithm to interact with the state directly, not only through opaque actions. 

Allowed assignments are limited by constraints in the problem and broadly speaking they can be divided into three groups:

* *Unary constraints* each concerning only a single variable (e.g., $x>0$)
* *Binary constraints* relating two variables to each other (e.g., $x>y$)
* *General constraints* concerning more than two variables at once

In general, the shape of constraints is unrestricted and we are not limited to numeric domains and constraints in the form of inequalities. In principle, one could employ arbitrary boolean functions as constraints.

The following class `CSP` is a very limited representation of such a problem capable only of handling:

* finite domains that can be materialized (e.g., the set of all 32-bit integers is doable in theory; the set of all 64-bit integers is out of reach) 
* unary constraints by setting the domain of a variable appropriately;
* binary constraints defined in terms of Python's operators `==`, `!=`, `<`, `<=`, `>` and `>=` such that both sides are variables.

About its methods:

* The constructor (`__init__`) requires no arguments.
* `addVariable` requires two arguments: 
    * `var` - a variable (a string representing its name, but really, it could be any object);
    * `domain` - a collection of the allowed values for the variable
* `addBinaryConstraint` also requires three arguments:
    * `var1` - a variable, the left-hand side of the constraint
    * `op` - an operator (one of the following strings: `'=='`, `'!='`, `'<'`, `'<='`, `'>'`, `'>='`)
    * `var2` - a variable, the right-hand side of the constraint
* `verify` is used to verify whether the relation given by the second argument `op` holds between the values given as the first argument `left` and the third argument `right`. Returns `True` if it does and `False` otherwise.
* `is_complete` gets a single argument: a dictionary representing an assignment of values to variables. From now on, we call such a dictionary an *assignment*. It returns `True` if all the variables of the problem are present in the dictionary, and `False` otherwise.
* `is_consistent` also requries an assignment. It returns `True` if all the values in the assignment are such that they come from the respecitve domains, and no binary constrain is violated (but if the assignment is not complete then some may be unfulfilled).

In [1]:
class CSP:
    def __init__(self):
        self.domains = {}
        self.binary = []
    
    def addVariable(self, var, domain):        
        assert var not in self.domains
        self.domains[var] = set(domain)
        
    def addBinaryConstraint(self, var1, operator, var2):
        assert var1 in self.domains
        assert var2 in self.domains
        c = (var1, operator, var2)
        self.binary.append(c)      
        
    def verify(self, left, op, right):
        if op[0] == '=':
            return left == right
        if op == '!=':
            return left != right
        if op == '<':
            return left < right
        if op == '<=':
            return left <= right
        if op == '>':
            return left > right
        if op == '>=':
            return left >= right
        
    def is_complete(self, assignment):
        return self.domains.keys() <= assignment.keys() 
        
    def is_consistent(self, assignment):
        for var, value in assignment.items():            
            if value not in self.domains[var]:
                return False
        for var1, op, var2 in self.binary:
            if var1 in assignment and var2 in assignment:
                if not self.verify(assignment[var1], op, assignment[var2]):
                    return False
        return True

## Coloring Australia

Lets model a map coloring problem. In particular, we'll color the following map of Australia. It can be simplified to a graph presented in the right-hand side of the picture below.

![](aima-images/fig_6_1.png)

We begin by creating a new problem in the variable `australia` over the domain consisting of three values: `'R'` statnding for the color red, `'G'` standing for green and `'B'` standing for blue. Of course the particular symbols used are not relevant. There are 7 teritories, so we add 7 variables to the problem.

In [2]:
australia = CSP()
australia.addVariable('WA', {'R', 'G', 'B'})
australia.addVariable('NT', {'R', 'G', 'B'})
australia.addVariable('SA', {'R', 'G', 'B'})
australia.addVariable('Q', {'R', 'G', 'B'})
australia.addVariable('NSW', {'R', 'G', 'B'})
australia.addVariable('V', {'R', 'G', 'B'})
australia.addVariable('T', {'R', 'G', 'B'})

Any two teritories of Australia sharing a border should be colored using different colours. We thus define 9 binary constraints.

In [3]:
australia.addBinaryConstraint('WA', '!=', 'NT')
australia.addBinaryConstraint('WA', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'NSW')
australia.addBinaryConstraint('SA', '!=', 'V')
australia.addBinaryConstraint('Q', '!=', 'NSW')
australia.addBinaryConstraint('NSW', '!=', 'V')

Let's see how the `verify` method works.

In [4]:
print("Is G=G?", australia.verify('G', '==', 'G'))
print("Is 5<7?", australia.verify(5, '<', 7))
print("Is 5>=7?", australia.verify(5, '>=', 7))

Is G=G? True
Is 5<7? True
Is 5>=7? False


Lets now consider an assignment such that *Western Australia* is red and the rest is unspecified. Is it a consistent assignment?

In [5]:
australia.is_consistent({'WA': 'R'})

True

Is it a complete assignment?

In [6]:
australia.is_complete({'WA': 'R'})

False

As expected it didn't work: there are still 9 unfulfiled constraints. So lets extend the solution.

In [7]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? False


Now it almost works: all the constraints are satisfied, but the solution is still not complete, as Tasmania is still colorless.

In [8]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G', 'T': 'B'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? True


Finally, we have manually collored the map of Australia! Now let's make computer do this for us.

## Task 1: Backtracking

Complete the following cell of code by implementing the recursive backtracking algorithm. Basically, it is a recursive DFS where an action is assigning a value to a variable. Avoid making copies of assignments, instead operate on a single assignment, and update it accordingly. Increment `self.counter` every time you check wheter an assigment is a solution (i.e., is consistent and complete). Use the provided methods `select_unassigned_variable` and `order_domain_values` to, respectively, get a next variable to assign a value to, and to get the list of allowed values for the variable. `solve` should return a solution (i.e., a complete, consistent assignment) or `None`.

In [9]:
class RecursiveBacktracking:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
        
    def select_unassigned_variable(self, assignment): #single element
        for var in self.csp.domains.keys():
            if var not in assignment:
                return var
        return None
    
    def order_domain_values(self, variable, assignment): #set
        return self.csp.domains[variable]

    def solve(self, assignment):
        if (self.csp.is_complete(assignment)):
            return assignment
        to_assign = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(to_assign, assignment):
            assignment[to_assign] = value
            self.counter +=1
            if (self.csp.is_consistent(assignment)):
                assignment = self.solve(assignment)
                if (assignment is not None):
                    return assignment
                assignment.pop(to_assign)
        return None

Let's test it on the Australia problem! In the cell below you should get a consistent, complete assignment, i.e., a solution to the problem.

In [10]:
solver = RecursiveBacktracking(australia)
assignment = solver.solve({})
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'WA': 'R', 'NT': 'G', 'SA': 'B', 'Q': 'R', 'NSW': 'G', 'V': 'R', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 11


## Task 2: Extend `RecursiveBacktracking` with the MRV and degree heuristics

Modify your `RecursiveBacktracking` to include the MRV (minimum remaining values) heuristics and the degree heuristics. Implement them in the appropriate places below in the class `RecursiveBacktrackingWithHeuristics`. If you want, you can also implement the least constraining value heuristics, but this is not mandatory. Follow the same assumptions as for `RecursiveBacktracking`, so expose the main backtracking algorithm as the `solve` and keep track of the number of considered assignments in the `counter` variable.

In [11]:
import copy

class RecursiveBacktrackingWithHeuristics:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
    
    def calculateNotAssignedNeighbours(self, assignment): #calculating neighbours which have not been assigned yet for each node
        neigh = {}
        nodes = list(assignment.keys())
        cons = self.csp.binary[:] #all constraints
        for i in self.csp.domains.keys():
            if (i in nodes):
                continue
            for left, op, right in cons:
                if (i not in neigh):
                    neigh[i] = []
                if (i == left and right not in assignment.keys()):
                    neigh[i].append(right)
                elif (i == right and left not in assignment.keys()):
                    neigh[i].append(left)
        return neigh
    
    def calculate_legal_values (self, assignment):
        new_domain = copy.deepcopy(self.csp.domains)
        for i in assignment.keys():
            new_domain.pop(i)
        neigh = self.calculateNotAssignedNeighbours({})
        for i in list(self.csp.domains.keys()):
            if (i in assignment.keys()):
                continue
            else:
                curr_neigh = neigh[i]
                for j in curr_neigh:
                    if (j in assignment.keys()):
                        new_domain[i].discard(assignment[j])
        return new_domain
        
    def calculate_mini(self, some_list):
        mini = len(some_list[0][1])
        list_of_minis = []
        for i in some_list:
            if(len(i[1]) < mini):
                mini = len(i[1])
                list_of_minis = [i]
            elif(len(i[1]) == mini):
                list_of_minis.append(i)
        return (mini, list_of_minis)
        
    def calculate_maxi(self,some_dict):
        maxi = 0
        list_of_maxis = []
        for i in some_dict.keys():
            if(len(some_dict[i]) > maxi):
                maxi = len(some_dict[i])
                list_of_maxis = [i]
            elif(len(some_dict[i]) ==maxi):
                list_of_maxis.append(i)
        return list_of_maxis[0]
    
    def select_unassigned_variable(self, assignment):
        unassigned = list(self.calculate_legal_values(assignment).items())
        new_unassigned = []
        for i, j in unassigned:
            new_unassigned.append((i, len(j)))
        mini, list_of_minis = self.calculate_mini(unassigned)
        if (len(list_of_minis) == 1): #MRV and finish
            return list_of_minis[0][0]
        list_of_minis = [list_of_minis[i][0] for i in range(len(list_of_minis))]
        const_on_current_variable = self.calculateNotAssignedNeighbours(assignment)
        all_keys = list(const_on_current_variable.keys())
        for i in all_keys:
            if (i not in list_of_minis):
                const_on_current_variable.pop(i)
        to_assign = self.calculate_maxi(const_on_current_variable)
        return to_assign
        
    def order_domain_values(self, variable, assignment):
        neigh = self.calculate_legal_values(assignment)
        return set(neigh[variable])
        
    def solve(self, assignment):
        if (self.csp.is_complete(assignment)):
            return assignment
        to_assign = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(to_assign, assignment):
            assignment[to_assign] = value
            self.counter +=1
            if (self.csp.is_consistent(assignment)):
                assignment = self.solve(assignment)
                if (assignment is not None):
                    return assignment
                assignment.pop(to_assign)
        return None

Lets test your implementation by solving the problem of coloring Australia!

In [12]:
solver = RecursiveBacktrackingWithHeuristics(australia)
assignment = solver.solve({})
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'R', 'NT': 'G', 'Q': 'B', 'NSW': 'G', 'WA': 'B', 'V': 'B', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 7


**Compare the number of considered assignments for `RecursiveBacktracking` and `RecursiveBacktrackingWithHeuristics`. How do they differ? Why do you think is that?**

RecursiveBactrackingWithHeuristics works better (7 assignments) than RecursiveBacktracking (11 assignments). The main difference between them is that RecursiveBacktracking assigns any value and then checks if it is correct and if not it backtracks. RecursiveBacktrackingWithHeuristics includes domain and checks which values are relevant and as a consequence it makes less assignments

## Sudoku!

Coloring Australia is really a bit boring. Lets deal with something more exciting: SUDOKU! I assume you know the rules, but if you don't, go look them up, spend some time solving a few puzzles and familiarize yourself with the puzzle.
The cell below defines a string `puzzle` which represents a Sudoku puzzle from the picture below. A digit in `puzzle` corresponds to a digit in the puzzle, while an underscore `_` corresponds to a blank cell in the puzzle. For your convenience there's also variable `solution` containing the correct solution to this particular puzzle.

![](aima-images/fig_6_4.png)

In [13]:
puzzle = '''
__3_2_6__
9__3_5__1
__18_64__
__81_29__
7_______8
__67_82__
__26_95__
8__2_3__9
__5_1_3__
'''

In [14]:
solution = '''
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382
'''

## Task 3: Sudoku as CSP

Using the string `puzzle` defined above write in the cell below code that will add all the necessary constraints to the CSP represented by the variable `sudoku`. If you can and want, write a parser that will construct the CSP from a string representation of the puzzle. Otherwise, you can construct the problem manually.

In [15]:
sudoku = CSP()

letters = [chr(i) for i in range(ord('A'), ord('A')+9)]
allNumbers = {'1', '2', '3', '4', '5', '6', '7', '8', '9'}

#adding domain:
for i in letters:
    for j in range(9):
        curr_tile = puzzle[(ord(i)-ord('A'))*10 + j + 1]
        id = str(i + str(j+1))
        if (curr_tile == '_'):
            sudoku.addVariable(id, allNumbers)
        else:
            sudoku.addVariable(id, curr_tile)

#adding constraints
for i in letters:
    for j in range (9):
        id = str(i+str(j+1))
        #in column
        for l in letters:
            new_id = str(l + id[1])
            if (new_id != id and (new_id, '!=', id) not in sudoku.binary and (id, '!=', new_id) not in sudoku.binary):
                sudoku.addBinaryConstraint(new_id, '!=', id)
        #in row
        for l in range(9):
            new_id = str(id[0] + str(l+1))
            if (new_id != id and (new_id, '!=', id) not in sudoku.binary and (id, '!=', new_id) not in sudoku.binary):
                sudoku.addBinaryConstraint(new_id, '!=', id)
        #define left upper corner of square:
        x = chr(((ord(i) - ord('A'))//3)*3 + ord('A'))
        y = (j//3)*3 + 1
        for l in letters[letters.index(x) : letters.index(x)+3]:
            for k in range (y, y+3):
                new_id = l+str(k)
                if (new_id != id and (new_id, '!=', id) not in sudoku.binary and (id, '!=', new_id) not in sudoku.binary):
                    sudoku.addBinaryConstraint(new_id, '!=', id)

Your `RecursiveBacktrackingWithHeuristics` should be able to handle this instance of sudoku with flying colors.

In [16]:
solver = RecursiveBacktrackingWithHeuristics(sudoku)
assignment = %time solver.solve({})
print("Assignment", assignment)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter) 

CPU times: total: 1.94 s
Wall time: 1.98 s
Assignment {'A3': '3', 'B4': '3', 'C7': '4', 'D6': '2', 'E1': '7', 'H9': '9', 'I5': '1', 'F6': '8', 'G3': '2', 'A7': '6', 'C4': '8', 'E9': '8', 'B1': '9', 'I3': '5', 'D4': '1', 'F7': '2', 'G6': '9', 'A5': '2', 'H1': '8', 'B9': '1', 'C3': '1', 'D7': '9', 'F4': '7', 'H6': '3', 'G7': '5', 'D3': '8', 'I7': '3', 'E7': '1', 'B6': '5', 'F3': '6', 'G4': '6', 'C6': '6', 'H4': '2', 'E6': '4', 'E3': '9', 'H7': '7', 'G9': '4', 'I4': '4', 'I1': '6', 'H5': '5', 'I9': '2', 'I8': '8', 'G8': '1', 'G1': '3', 'G2': '7', 'I2': '9', 'A4': '9', 'C5': '7', 'B5': '4', 'B3': '7', 'B8': '2', 'H8': '6', 'A6': '1', 'B7': '8', 'B2': '6', 'E4': '5', 'E8': '3', 'E2': '2', 'C2': '5', 'F9': '5', 'F8': '4', 'A1': '4', 'D1': '5', 'A2': '8', 'C8': '9', 'C9': '3', 'D8': '7', 'F1': '1', 'G5': '8', 'D9': '6', 'D5': '3', 'F2': '3', 'A8': '5', 'D2': '4', 'E5': '6', 'H2': '1', 'A9': '7', 'C1': '2', 'F5': '9', 'H3': '4', 'I6': '7'}
Is consistent? True
Is complete? True
# considered ass

------------
The pictures are from "Artificial Intelligence: A Modern Approach" 3rd ed.